In [1]:
import os
import json
import os.path
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir('..')
from dotenv import load_dotenv

load_dotenv(".env")  # take environment variables from .env.

from steam_trade_bot.etl.settings import get_jdbc_creds

In [2]:
%%time
from steam_trade_bot.etl.spark_conf import create_spark_instance

spark = create_spark_instance()

CPU times: total: 188 ms
Wall time: 15.5 s


In [3]:
%%time
jdbc_url, username, password = get_jdbc_creds()
df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "stg_market.market_item_sell_history") \
    .option("user", username) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .load()

CPU times: total: 15.6 ms
Wall time: 3.55 s


In [4]:
df.show()

+------+--------------------+--------------------+--------------------+
|app_id|    market_hash_name|           timestamp|             history|
+------+--------------------+--------------------+--------------------+
|   730|M4A1-S | Nitro (F...|2022-11-12 19:16:...|"[[1385676000.0, ...|
|   730|M4A1-S | Boreal F...|2022-11-12 18:48:...|"[[1385589600.0, ...|
|   730|M4A1-S | Fizzy PO...|2022-11-12 18:49:...|"[[1632261600.0, ...|
|   730|M4A1-S | Boreal F...|2022-11-12 20:00:...|"[[1385589600.0, ...|
|   730|M4A1-S | VariCamo...|2022-11-12 18:50:...|"[[1385589600.0, ...|
|   730|M4A1-S | Cyrex (M...|2022-11-12 20:00:...|"[[1404252000.0, ...|
|   730|M4A1-S | Boreal F...|2022-11-12 18:46:...|"[[1385589600.0, ...|
|   730|M4A1-S | Nitro (F...|2022-11-12 18:45:...|"[[1385589600.0, ...|
|   730|M4A1-S | VariCamo...|2022-11-12 18:44:...|"[[1385676000.0, ...|
|   730|M4A1-S | Boreal F...|2022-11-12 18:45:...|"[[1385589600.0, ...|
|   730|M4A1-S | Nitro (W...|2022-11-12 18:53:...|"[[1385589600.

In [5]:
df.printSchema()

root
 |-- app_id: integer (nullable = true)
 |-- market_hash_name: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- history: string (nullable = true)



In [6]:
from pyspark.sql.functions import from_json, col, regexp_replace, explode, to_timestamp
import pyspark.sql.functions as func
from pyspark.sql.types import ArrayType, IntegerType, FloatType, DecimalType

# json_schema = spark.read.json(df.rdd.map(lambda row: row.history)).schema
df2 = df.withColumn("history", regexp_replace(col("history"), '"', ""))
df3 = df2.withColumn('history', from_json(col('history'), ArrayType(ArrayType(DecimalType(18, 2)))))
df4 = df3.select('app_id', 'market_hash_name', 'timestamp', explode('history').alias('exploded_history'))
df5 = df4.select(
    col("app_id"),
    col("market_hash_name"),
    col("timestamp"),
    to_timestamp(col('exploded_history')[0].cast('bigint')).alias("point_timestamp"),
    col("exploded_history")[1].alias("price"),
    col("exploded_history")[2].cast('integer').alias("sold_quantity")
).repartition("market_hash_name")

In [7]:
df5.first()

Row(app_id=730, market_hash_name='M4A1-S | Nitro (Well-Worn)', timestamp=datetime.datetime(2022, 11, 12, 18, 53, 36, 406543), point_timestamp=datetime.datetime(2013, 11, 28, 1, 0), price=Decimal('26.38'), sold_quantity=2)

In [8]:
df5.show(20, False)

+------+--------------------------+--------------------------+-------------------+-----+-------------+
|app_id|market_hash_name          |timestamp                 |point_timestamp    |price|sold_quantity|
+------+--------------------------+--------------------------+-------------------+-----+-------------+
|730   |M4A1-S | Nitro (Well-Worn)|2022-11-12 18:53:36.406543|2013-11-28 02:00:00|26.38|2            |
|730   |M4A1-S | Nitro (Well-Worn)|2022-11-12 18:53:36.406543|2013-11-29 02:00:00|19.85|7            |
|730   |M4A1-S | Nitro (Well-Worn)|2022-11-12 18:53:36.406543|2013-11-30 02:00:00|18.56|3            |
|730   |M4A1-S | Nitro (Well-Worn)|2022-11-12 18:53:36.406543|2013-12-01 02:00:00|19.17|5            |
|730   |M4A1-S | Nitro (Well-Worn)|2022-11-12 18:53:36.406543|2013-12-02 02:00:00|13.53|7            |
|730   |M4A1-S | Nitro (Well-Worn)|2022-11-12 18:53:36.406543|2013-12-03 02:00:00|16.05|2            |
|730   |M4A1-S | Nitro (Well-Worn)|2022-11-12 18:53:36.406543|2013-12-04 

In [9]:
df5.count()

300328

In [10]:
df5.rdd.getNumPartitions()

3

In [11]:
df6 = df5.withColumn("point_timestamp",func.date_trunc("day", col("point_timestamp"))).groupBy("point_timestamp").agg(
    func.round(func.avg((col("price"))), 2).alias("daily_avg_price"),
    func.sum((col("price") * col("sold_quantity"))).alias("daily_volume"),
    func.sum(col("sold_quantity")).alias("daily_quantity"),
    func.approx_count_distinct("market_hash_name").alias("sold_unique_items"),
).sort("point_timestamp", ascending=False)
df6.show(20, False)

+-------------------+---------------+------------+--------------+-----------------+
|point_timestamp    |daily_avg_price|daily_volume|daily_quantity|sold_unique_items|
+-------------------+---------------+------------+--------------+-----------------+
|2022-11-12 00:00:00|15.59          |48872.02    |9555          |105              |
|2022-11-11 00:00:00|15.75          |75734.06    |15062         |109              |
|2022-11-10 00:00:00|15.03          |68997.43    |13555         |108              |
|2022-11-09 00:00:00|14.43          |67615.85    |13368         |108              |
|2022-11-08 00:00:00|14.43          |65245.23    |12874         |106              |
|2022-11-07 00:00:00|14.48          |66884.06    |13272         |108              |
|2022-11-06 00:00:00|17.10          |81804.01    |16127         |108              |
|2022-11-05 00:00:00|14.32          |75947.00    |16535         |108              |
|2022-11-04 00:00:00|14.54          |75344.48    |15553         |106        

In [12]:
df6.count()

3377

In [13]:
df6.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "stg_market.entire_market_daily_stats") \
    .option("user", username) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .option("truncate", True) \
    .save()

In [14]:
# TODO: compute top100 items sold for each day

In [15]:
# TODO: boost detect on daily basis, write to separate table like `boosted_items`
# looks like this task is for each item, like first job